In [1]:
import pandas as pd
import numpy as np
import altair as alt
import eco_style
import json
alt.themes.enable('light')

gh_root = "https://raw.githubusercontent.com/jhellingsdata/RADataHub/main/Chart%20Packs/Housing/"

# OECD House Prices

In [2]:
df = pd.read_csv('oecd_data.csv')
df = df[df.VINTAGE == 'VINTAGE_TOTAL']
df = df[['REG_ID', 'Region', 'TIME', 'Value']]
df.columns = ['iso3', 'country', 'date', 'value']
df = df[df.iso3.apply(lambda x: len(x) == 3)]
# drop everything with brackets in its country (these are subnational regions)
df = df[~df.country.str.contains('\(')]

iso3s = df.query("date == '2005-Q1'").iso3.unique()
df = df[df.iso3.isin(iso3s)]

exclude = [ # We just want deveoloped OECD countries
    "CHL", "BRA"
]

df = df[~df.iso3.isin(exclude)]

# reindex to 2005=100
df = df.merge(df.query("date == '2010'"), on='iso3', suffixes=('', '_2005'))
df['value'] = df.value / df.value_2005 * 100

df = df[df.date.str.contains("-")]

def quarters_to_date(date_str):
    year = date_str.split("-")[0]
    quarter = int(date_str.split("-")[1][-1])
    month = (quarter - 1) * 3 + 1
    return f"{year}-{month:02d}-01"

df['date'] = df.date.apply(quarters_to_date)
df = df[['iso3', 'country', 'date', 'value']]

df.drop_duplicates(subset=['iso3'], keep='last').sort_values('value', ascending=False)

,iso3,country,date,value
1698,ISL,Iceland,2023-07-01,320.023605
871,EST,Estonia,2023-07-01,310.264315
1582,HUN,Hungary,2023-04-01,289.992564
2032,MEX,Mexico,2023-07-01,246.547708
2344,NZL,New Zealand,2023-07-01,233.197986
1939,ISR,Israel,2023-07-01,212.828740
674,AUT,Austria,2023-07-01,212.297128
2437,NOR,Norway,2023-07-01,194.350954
2530,SWE,Sweden,2023-07-01,179.287501
302,AUS,Australia,2021-10-01,178.024655


# Britain vs France Housing Starts

In [3]:
df = pd.read_csv("fra_starts.csv", skiprows=3, sep=";", encoding="latin1")
df = df.iloc[:, :-1]
df.columns = ['date', 'value']
df['date']  = pd.to_datetime(df.date).dt.strftime("%Y-%m-%d")
df['country'] = 'France'
fra_df = df.copy()

df = pd.read_excel("ukhousebuilding-2.xlsx", sheet_name="1a", skiprows=5)
df = df[['Period', 'Started - All Dwellings']]
df.columns = ['date', 'value']
df['month'] = ['01', '04', '07', '10']*int(len(df)/4)
df['year'] = df.date.str[-4:]
df['date'] = df.year + '-' + df.month + '-01'
df['date'] = pd.to_datetime(df.date)

# drop everything between Apr 2020 and Oct 2022
df = df[~((df.date >= '2020-04-01') & (df.date <= '2022-10-01'))]

df['date'] = df.date.dt.strftime("%Y-%m-%d")

df['value'] = pd.to_numeric(df.value, errors='coerce')
df = df.dropna(subset=['value'])
df['value'] = df.value.rolling(4).sum()
df['country'] = 'UK'
df = df[['country', 'date', 'value']]
uk_df = df.copy()

df = pd.read_csv("canada.csv")
df = df[(df['Housing estimates'] == 'Housing starts') & (df['GEO'] == 'Canada') & (df['SCALAR_FACTOR'] == 'units') & (df['Type of unit'] == 'Total units')]
df['date'] = df['REF_DATE']
df['date'] = pd.to_datetime(df.date).dt.strftime("%Y-%m-%d")
df['value'] = pd.to_numeric(df.VALUE, errors='coerce')
df = df[['date', 'value']]
# group by year 
df['year'] = pd.to_datetime(df.date).dt.year
df = df.groupby('year').sum().reset_index()
df['country'] = 'Canada'
df = df.query("year < 2024")
df['date'] = df.year.astype(str) + '-01-01'
df = df[['country', 'date', 'value']]
canada_df = df.copy()

df = pd.concat([canada_df, uk_df, fra_df])
df

full_housing_panel = df.copy()

In [4]:

df = df.sort_values('date')
df.drop_duplicates(subset=['country'], keep='last')

,country,date,value
75,Canada,2023-01-01,240267.0
183,UK,2023-10-01,176460.0
0,France,2024-04-01,282400.0


In [5]:

# add labels at the end for each 

df = df.query("date <= '2023-10-01'")
df['label'] = np.where(df.date == df.date.max(), df.country, '')


df = df.query("country != 'Canada' and date >= '2000-01-01'")
df.to_csv("uk_fra_housing_starts_clean.csv", index=False)

df_url = gh_root + "uk_fra_housing_starts_clean.csv"

base = alt.Chart(
    alt.Data(url=df_url)
).encode(
    x=alt.X('date:T', title=None),
    y=alt.Y('value:Q', title=None),
    strokeDash=alt.condition(
        alt.datum.country == 'France',
        alt.value([3, 3]),
        alt.value([0, 0])
    ),
    color=alt.Color('country:N', 
                    legend=None,
                    title=None),
)

line = base.mark_line(point={
    "size": 0,
})

label = base.mark_text(
    align='left',
    size=12,
    dx=5,
    dy=0).encode(
    text='label:N'
)

chart = line + label

chart.save("uk_vs_france_headless.json")

chart = chart.properties(
        width=400,
        height=300,
        title={
        "text": ["Housebuilding in the UK and France"],
        "dx": 50,
        "anchor": "start",
        "subtitle": ["Housing Starts, not seasonally adjusted", "Source: ONS, INSEE", ""],
        "subtitleColor": "#676A86",
    }
)

chart.save("uk_vs_france.svg", scale_factor=2.0)
chart.save("uk_vs_france.png", scale_factor=2.0)
chart.save("uk_vs_france.json")
chart

WARN strokeDash dropped as it is incompatible with "text".
WARN strokeDash dropped as it is incompatible with "text".


alt.LayerChart(...)

In [308]:
df.query("country == 'UK'")

,country,date,value,label
0,UK,1978-01-01,NaN,
1,UK,1978-04-01,NaN,
2,UK,1978-07-01,NaN,
3,UK,1978-10-01,272470.0,
4,UK,1979-01-01,257240.0,
...,...,...,...,...
168,UK,2020-01-01,187490.0,
180,UK,2023-01-01,183500.0,
181,UK,2023-04-01,209330.0,
182,UK,2023-07-01,198490.0,


In [310]:
df

,country,date,value,label
0,Canada,1948-01-01,90194.0,
1,Canada,1949-01-01,90509.0,
2,Canada,1950-01-01,92531.0,
3,Canada,1951-01-01,68579.0,
4,Canada,1952-01-01,83246.0,
...,...,...,...,...
9,France,2023-07-01,341100.0,
8,France,2023-08-01,335400.0,
7,France,2023-09-01,327000.0,
183,UK,2023-10-01,176460.0,UK


# UK vs Tokyo

In [338]:
data = [{
    "area": "UK",
    "value": "176460.0"
},
{
    "area": "Tokyo",
    "value": "129552"
}
]

df = pd.DataFrame(data)
base = alt.Chart(df).encode(
    y=alt.Y('area:N', 
            axis=alt.Axis(ticks=False, domain=False, title=None, labelColor="black", labelFontSize=10),
            title=None),
    x=alt.X('value:Q', title=None)
)

bars = base.mark_bar(color="#36b7b4")

text = base.mark_text(
    align='left',
    dx=3
).encode(
    text=alt.Text('value:Q', format=",.0f")
)


chart = (bars + text).properties(
        height=200,
        width=300,
        title={
        "text": ["Housebuilding in the UK and Tokyo"],
        "dx": 35,
        "anchor": "start",
        "subtitle": ['Gross housing starts', "Source: ONS, e-stat", ""],
        "subtitleColor": "#676A86",
    }
)

chart.save("uk_vs_tokyo.svg")
chart.save("uk_vs_tokyo.png", scale_factor=2.0)
chart

/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.LayerChart(...)

In [309]:
198490.0-176460.0

22030.0

# 10 year comparison

In [186]:
df = full_housing_panel.copy()
df = df.query("country != 'Canada' and date >= '2011-10-01'")
df = df[df.date.astype(str).str.contains("10-01")]
df = df.pivot(index='date', columns='country', values='value')
df = df.reset_index()
df = df.dropna()
df['shortfall'] = df['France'] - df['UK']
df.shortfall.sum()

2037360.0

# Long-run UK

,0,1
0,1958.345865,0.000000
1,1956.390977,1.581308
2,1955.338346,1.132710
3,1954.285714,1.059813
4,1950.375940,2.725234
...,...,...
68,1864.210526,0.588785
69,1863.007519,0.583178
70,1862.105263,0.661682
71,1861.052632,0.661682


In [28]:
public_df = pd.read_csv("picked/public.csv", header=None)
private_df = pd.read_csv("picked/private.csv", header=None)

public_df.columns = ['date', 'value']
private_df.columns = ['date', 'value']

public_df['series'] = 'Public'
private_df['series'] = 'Private'

df = pd.concat([public_df, private_df])
df['date'] = np.round(df.date, 0).astype(int).astype(str) + "-01-01"

df = df.drop_duplicates(subset=['date', 'series'], keep='last')

df = df.pivot(index='date', columns='series', values='value').reset_index()

df = df.dropna(subset=['Public', 'Private'])
df['Public'] = df.Public - df.Private

df = df.melt(id_vars='date', value_vars=['Public', 'Private'])
df['value'] = df.value/100


In [29]:

df.to_csv("england_wales_longrun.csv", index=False)

df_url = gh_root + "england_wales_longrun.csv"

# stacked area chart
chart = alt.Chart(
    alt.Data(url=df_url)
).mark_area(
    interpolate='step-after',
).encode(
    x=alt.X('date:T', title=None),
    y=alt.Y('value:Q', title="Gross housebuilding, % of existing stock", 
            axis=alt.Axis(
                titleY=-25,
                format="%"),
            scale=alt.Scale(domainMin=0)),
    color=alt.Color('series:N',
                    legend=alt.Legend(
                        orient='none',
                        direction='horizontal',
                        legendX=0,
                        legendY=-20

                    ),
                     title=None),
    strokeDash=alt.condition(
        alt.datum.series == 'Public',
        alt.value([3, 3]),
        alt.value([0, 0])
    ),
)

chart = chart.properties(
    width=500,
    height=300,
)

chart.save("england_wales_headless.json")
chart


alt.Chart(...)

In [ ]:

chart = chart.properties(
        title={
        "text": ["Housebuilding in England and Wales"],
        "dx": 30,
        "anchor": "start",
        "subtitle": ["Gross housebuilding each year relative to existing stock", "Source: Waitling and Breach (2023) based on Holman (2005)", ""],
        "subtitleColor": "#676A86",
    }
)

chart.save("england_wales.png", scale_factor=2.0)
chart.save("england_wales.json")
chart

# Longrun-redo

In [2]:
public_df = pd.read_csv("picked/public.csv", header=None)
private_df = pd.read_csv("picked/private.csv", header=None)

public_df.columns = ['date', 'value']
private_df.columns = ['date', 'value']

public_df['series'] = 'Public'
private_df['series'] = 'Private'

df = pd.concat([public_df, private_df])
df['value'] = df.value/100

# turn fractional years into dates
def fractional_to_date(frac):
    year = int(frac)
    days = (frac - year) * 365
    return pd.to_datetime(f"{year}-01-01") + pd.DateOffset(days=days)

df['date'] = df.date.apply(fractional_to_date)

private_layer = alt.Chart(df.query("series == 'Private'")).mark_area(color="#36b7b4").encode(
    x=alt.X('date:T', title=None),
    y=alt.Y('value:Q', title="Gross housebuilding, % of existing stock", 
            axis=alt.Axis(
                titleY=-25,
                format="%"),
            scale=alt.Scale(domainMin=0)),
    color=alt.Color('series:N', 
                    legend=alt.Legend(orient='none', 
                                      direction='horizontal',
                                      legendY=-20,
                                      
                                      title=None),
                    scale=alt.Scale(domain=['Private', 'Public'], range=["#36b7b4", "#E6224B"])),
)

public_layer = alt.Chart(df.query("series == 'Public'")).mark_area(color="#E6224B").encode(
    x=alt.X('date:T', title=None),
    y=alt.Y('value:Q', title="Gross housebuilding, % of existing stock", 
            axis=alt.Axis(
                titleY=-25,
                format="%"),
            scale=alt.Scale(domainMin=0)),
)

chart = public_layer + private_layer
chart

chart = chart.properties(
        width=500,
        height=300,
)

chart.save("england_wales_headless.json")
chart.save("england_wales.svg")
chart.save("england_wales.png", scale_factor=2.0)

chart

/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/

alt.LayerChart(...)

In [42]:
public_layer

/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [36]:
df.date.value_counts()

date
1958-05-07    2
1866-01-06    2
1915-01-14    2
1916-02-02    2
1917-02-22    2
             ..
2020-01-01    1
2019-02-05    1
2017-02-22    1
2016-03-28    1
1864-01-22    1
Name: count, Length: 168, dtype: int64

In [15]:
df_url

'https://raw.githubusercontent.com/jhellingsdata/RADataHub/main/Chart%20Packs/Housing/england_wales_longrun.csv'

# Japan

In [213]:
dfs = pd.read_html("jpn_stats.html")
df = dfs[4].copy()

df.columns = df.columns.droplevel(0)
df.columns = df.columns.droplevel(0)
df = df.iloc[:, :2]
df.columns = ['area', 'value']
df.to_csv("jpn.csv", index=False)

In [192]:
df.columns

Index(['date', ' value', 'series'], dtype='object')

In [177]:

df['value'] = pd.to_numeric(df.value, errors='coerce')
df = df.dropna()
df['country'] = 'UK'
df


,country,date,value
143,UK,2013-10-01,150120.0
144,UK,2014-01-01,162210.0
145,UK,2014-04-01,167330.0
146,UK,2014-07-01,170200.0
147,UK,2014-10-01,170120.0
...,...,...,...
122,UK,2014-02-01,350000.0
123,UK,2014-01-01,356500.0
124,UK,2013-12-01,357900.0
125,UK,2013-11-01,369600.0


In [214]:


df = pd.concat([fra_df, uk_df])
df = df.query("date >= '2001-01-01'")

base = alt.Chart(df).encode(
    x=alt.X('date:T', axis=alt.Axis(title='')),
    y=alt.Y('value:Q', axis=alt.Axis(title='Index (2005=100)')),
    color=alt.Color('country:N')
)

lines = base.mark_line()

chart = lines

chart

/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

# UK Housing by LAD

In [7]:
df = pd.read_excel("housing_lads.xlsx", sheet_name="UK_Starts", skiprows=5)
df = df.melt(id_vars=['Local Authority Code', 'Region or Country Name',
       'Local Authority Name'], var_name='date', value_name='value')
df = df.dropna(subset=['value'])
df['date'] = df['date'].astype(str).str.split("-").str[0]
df.columns = ['id', 'region', 'name', 'date', 'starts']
df['starts'] = pd.to_numeric(df.starts, errors='coerce')
df = df.dropna(subset=['starts'])
starts_df = df.copy()

df = pd.read_csv("lad_pop.csv", skiprows=6)
df.columns = ["name", "id"] + list(df.columns[2:])
df = df.melt(id_vars=['name', 'id'], var_name='date', value_name='population')
df = df.query("date >= '2009'")
df = df.iloc[:-4, :]
# fill NA with previous value (missing 2022 NI estimates)
df = df.sort_values(['id', 'date'])
df['population'] = pd.to_numeric(df.population, errors='coerce')
df['population'] = df.population.fillna(method='ffill')
df = df.dropna(subset=['id'])
pop_df = df.copy()

df = starts_df.merge(pop_df[['id', 'date', 'population']], on=['id', 'date'], how='left')
df['value'] = df.starts / df.population * 1000
df.to_csv("lad_starts_per_1K.csv", index=False)

df[['id', 'region', 'name', 'date', 'value']].pivot_table(index=['id', 'region', 'name'], columns='date', values='value').reset_index().to_csv("lad_starts_per_1K_wide.csv", index=False)

/var/folders/07/g_yq2xsn0mvb4xnyvr18jjmw0000gn/T/ipykernel_71644/3062878682.py:19: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['population'] = df.population.fillna(method='ffill')


In [9]:
df.query("date == '2022' and name == 'Islington'")

,id,region,name,date,starts,population,value
4898,E09000019,London,Islington,2022,30.0,220373.0,0.136133


In [353]:
df.query("date == '2022' and region == 'London'").sort_values('starts', ascending=False).tail(11).starts.sum()

1210.0

In [339]:
df

,area,value
0,UK,176460.0
1,Tokyo,129552


In [298]:
starts_df.region.value_counts()

region
South East                  886
East of England             620
North West                  537
East Midlands               480
Scotland                    448
London                      436
West Midlands               409
South West                  399
Wales                       308
Yorkshire and The Humber    293
North East                  167
Northern Ireland             88
Name: count, dtype: int64

# London

In [299]:
london_lads = starts_df.query("region == 'London'").id.unique()
london_lads

array(['E09000002', 'E09000003', 'E09000004', 'E09000006', 'E09000007',
       'E09000008', 'E09000009', 'E09000010', 'E09000011', 'E09000012',
       'E09000013', 'E09000015', 'E09000016', 'E09000017', 'E09000018',
       'E09000019', 'E09000020', 'E09000021', 'E09000023', 'E09000026',
       'E09000029', 'E09000030', 'E09000031', 'E09000001', 'E09000022',
       'E09000024', 'E09000005', 'E09000014', 'E09000025', 'E09000027',
       'E09000028', 'E09000032', 'E09000033'], dtype=object)

In [303]:
uk_lads = json.load(open("Local_Authority_Districts_December_2021_GB_BGC_2022_4485095093561745628.geojson"))
uk_lads['features'] = [f for f in uk_lads['features'] if f['properties']['LAD21CD'] in london_lads]
with open("london_lads_BGC.geojson", "w") as f:
    json.dump(uk_lads, f)

In [294]:
df.value.quantile(0.95)

6.305978576189007

date,id,region,name,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,E06000001,North East,Hartlepool,2.512837,2.506184,2.063244,1.407780,3.028173,3.897876,2.930801,3.034901,2.708383,2.817268,1.731583,3.579098,5.293006,6.285891
1,E06000002,North East,Middlesbrough,2.039731,NaN,1.445421,1.660986,3.679972,4.027908,3.581611,2.488464,3.328305,4.324493,2.954958,3.790564,4.793564,3.237010
2,E06000003,North East,Redcar and Cleveland,1.766433,1.551155,2.071558,1.556120,3.040686,2.368511,3.545837,3.103204,3.097711,3.008600,2.706677,3.148545,2.854947,3.717879
3,E06000004,North East,Stockton-on-Tees,2.316058,2.619145,2.919343,2.541006,2.784610,2.767627,2.604832,4.276724,4.364463,5.214952,2.946256,1.985865,2.994437,4.000680
4,E06000005,North East,Darlington,1.245748,2.189892,1.041825,0.947355,2.935856,3.874321,2.264407,4.234975,4.129865,5.074663,4.693425,4.025200,4.066168,4.750203
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369,W06000020,Wales,Torfaen,2.412307,1.757083,1.425595,1.535105,1.651219,2.201043,1.431025,2.424777,3.089212,4.828267,2.944866,2.165979,1.730590,1.292268
370,W06000021,Wales,Monmouthshire,2.650264,3.296124,3.059842,2.837715,2.395940,2.721414,2.830178,3.368686,2.505692,2.395627,4.142592,3.569265,3.220163,2.662804
371,W06000022,Wales,Newport,2.782551,1.588365,2.195013,2.248569,3.658884,5.399933,6.176652,5.631899,3.915580,2.641378,1.786022,1.767253,2.191788,2.662440
372,W06000023,Wales,Powys,2.404388,2.483481,2.254436,1.877638,1.356249,1.355330,1.278724,1.358173,0.452967,0.303106,1.824318,0.907722,2.621311,1.941878


In [271]:
pop_df.query("id == 'N09000011'")

,name,id,date,population
373,Ards and North Down,N09000011,1991,-
751,Ards and North Down,N09000011,1992,-
1129,Ards and North Down,N09000011,1993,-
1507,Ards and North Down,N09000011,1994,-
1885,Ards and North Down,N09000011,1995,-
2263,Ards and North Down,N09000011,1996,-
2641,Ards and North Down,N09000011,1997,-
3019,Ards and North Down,N09000011,1998,-
3397,Ards and North Down,N09000011,1999,-
3775,Ards and North Down,N09000011,2000,-


In [269]:
starts_df

,id,region,name,date,starts
0,E06000001,North East,Hartlepool,2009,230
1,E06000002,North East,Middlesbrough,2009,280
2,E06000003,North East,Redcar and Cleveland,2009,240
3,E06000004,North East,Stockton-on-Tees,2009,440
4,E06000005,North East,Darlington,2009,130
...,...,...,...,...,...
5231,N09000007,Northern Ireland,Lisburn and Castlereagh,2022,490
5232,N09000008,Northern Ireland,Mid and East Antrim,2022,520
5233,N09000009,Northern Ireland,Mid Ulster,2022,470
5234,N09000010,Northern Ireland,"Newry, Mourne and Down",2022,600


In [268]:
pop_df

,name,id,date,population
0,Darlington,E06000005,1991,99342
1,County Durham,E06000047,1991,501488
2,Hartlepool,E06000001,1991,91058
3,Middlesbrough,E06000002,1991,144747
4,Northumberland,E06000057,1991,305521
...,...,...,...,...
12091,Ards and North Down,N09000011,2022,-
12092,-,These figures are missing.,2022,NaN
12093,NaN,"Mid-2022 estimates for United Kingdom, Great B...",2022,NaN
12094,NaN,Ireland are not yet available. These will be r...,2022,NaN


In [73]:
fra_df

,date,value,country
0,2024-04-01,282400,France
1,2024-03-01,282600,France
2,2024-02-01,286400,France
3,2024-01-01,289800,France
4,2023-12-01,298100,France
...,...,...,...
276,2001-04-01,336900,France
277,2001-03-01,333500,France
278,2001-02-01,337700,France
279,2001-01-01,340100,France


In [67]:
df.date.iloc[0]

1980

In [56]:
df.columns

Index(['Revised', 'Period', 'Started - All Dwellings',
       'Started - Private Enterprise', 'Started - Housing Associations',
       'Started - Local Authorities', 'Completed - All Dwellings',
       'Completed - Private Enterprise', 'Completed - Housing Associations',
       'Completed - Local Authorities'],
      dtype='object')

In [39]:
df.iso3.value_counts()

iso3
GBR    276
HUN    250
NZL    219
AUT    186
ISR    148
CHE    118
DEU    118
FRA    118
ISL    116
DNK    104
BEL     93
SWE     93
NOR     93
IRL     93
NLD     93
MEX     93
FIN     93
EST     93
POL     93
AUS     92
Name: count, dtype: int64

In [34]:
df.query("iso3 == 'GBR'")

,iso3,country,date,value,country_2005,date_2005,value_2005
2095,GBR,United Kingdom,1969,2.453420,United Kingdom,2005,78.951
2096,GBR,United Kingdom,1970,2.596547,United Kingdom,2005,78.951
2097,GBR,United Kingdom,1971,2.905600,United Kingdom,2005,78.951
2098,GBR,United Kingdom,1972,3.883421,United Kingdom,2005,78.951
2099,GBR,United Kingdom,1973,5.342554,United Kingdom,2005,78.951
...,...,...,...,...,...,...,...
2366,GBR,United Kingdom,2022-Q3,185.698725,United Kingdom,2005,78.951
2367,GBR,United Kingdom,2022-Q4,186.145837,United Kingdom,2005,78.951
2368,GBR,United Kingdom,2023-Q1,181.793771,United Kingdom,2005,78.951
2369,GBR,United Kingdom,2023-Q2,181.712708,United Kingdom,2005,78.951


In [33]:
df.date.unique()

array(['2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012',
       '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020',
       '2021', '2022', '2005-Q1', '2005-Q2', '2005-Q3', '2005-Q4',
       '2006-Q1', '2006-Q2', '2006-Q3', '2006-Q4', '2007-Q1', '2007-Q2',
       '2007-Q3', '2007-Q4', '2008-Q1', '2008-Q2', '2008-Q3', '2008-Q4',
       '2009-Q1', '2009-Q2', '2009-Q3', '2009-Q4', '2010-Q1', '2010-Q2',
       '2010-Q3', '2010-Q4', '2011-Q1', '2011-Q2', '2011-Q3', '2011-Q4',
       '2012-Q1', '2012-Q2', '2012-Q3', '2012-Q4', '2013-Q1', '2013-Q2',
       '2013-Q3', '2013-Q4', '2014-Q1', '2014-Q2', '2014-Q3', '2014-Q4',
       '2015-Q1', '2015-Q2', '2015-Q3', '2015-Q4', '2016-Q1', '2016-Q2',
       '2016-Q3', '2016-Q4', '2017-Q1', '2017-Q2', '2017-Q3', '2017-Q4',
       '2018-Q1', '2018-Q2', '2018-Q3', '2018-Q4', '2019-Q1', '2019-Q2',
       '2019-Q3', '2019-Q4', '2020-Q1', '2020-Q2', '2020-Q3', '2020-Q4',
       '2021-Q1', '2021-Q2', '2021-Q3', '2021-Q4', '2022-Q1',

In [26]:
iso3s

array(['BEL', 'FRA', 'AUS', 'AUT', 'CHL', 'DNK', 'EST', 'FIN', 'DEU',
       'HUN', 'ISL', 'IRL', 'ISR', 'MEX', 'NLD', 'NZL', 'NOR', 'SWE',
       'GBR', 'CHE', 'POL', 'BRA'], dtype=object)

In [18]:
df.date.value_counts()

date
2017-Q1    39
2018-Q1    39
2014-Q3    39
2014-Q4    39
2015-Q1    39
           ..
1971-Q1     1
1971-Q2     1
1971-Q3     1
1971-Q4     1
1979-Q3     1
Name: count, Length: 276, dtype: int64

In [13]:
df.query("country == 'United States'")

,iso3,country,date,value


In [15]:
df.iso3.unique()

array(['BEL', 'FRA', 'AUS', 'AU1', 'AU2', 'AU3', 'AU4', 'AU5', 'AU6',
       'AU7', 'AU8', 'AUT', 'CHL', 'CZE', 'DNK', 'EST', 'FIN', 'DEU',
       'HUN', 'ISL', 'IRL', 'ISR', 'ITA', 'ITC', 'ITH', 'ITI', 'JPN',
       'JPA', 'JPB', 'JPC', 'JPE', 'JPG', 'JPH', 'JPI', 'JPJ', 'LVA',
       'LTU', 'LUX', 'MEX', 'NLD', 'NZL', 'NOR', 'PRT', 'SVK', 'SVN',
       'ESP', 'SWE', 'TUR', 'GBR', 'UKC', 'UKD', 'UKE', 'UKF', 'UKG',
       'UKH', 'UKI', 'UKJ', 'UKK', 'UKL', 'UKM', 'UKN', 'GRC', 'CHE',
       'CHN', 'SAU', 'IND', 'POL', 'BRA'], dtype=object)